In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import train_test_split

import gc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'v1.py', 'test.csv', 'train_sample.csv', 'submission.csv', '.gitignore', 'train.csv', '.ipynb_checkpoints', 'v1.ipynb', '.git']


In [2]:
path = './'

train_skip = range(1, 140000000)
train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8'
        }

In [3]:
print("loading training data")
train= pd.read_csv(path + "train.csv", skiprows=train_skip, dtype=dtypes, header=0, usecols=train_cols)
# test = pd.read_csv(path + "test.csv", dtype=dtypes, header=0, usecols=test_cols)

loading training data


In [4]:
train['hour'] = pd.to_datetime(train.click_time).dt.hour.astype('uint8')
train['day'] = pd.to_datetime(train.click_time).dt.day.astype('uint8')

In [5]:
n_chans = train[['ip','day','hour','channel']].groupby(by=['ip','day',
          'hour'])[['channel']].count().reset_index().rename(columns={'channel': 'n_channels'})

In [6]:
train = train.merge(n_chans, on=['ip', 'day', 'hour'], how='left')

In [7]:
train = train.drop(['click_time', 'ip'], axis=1)

In [8]:
train['n_channels'] = train['n_channels'].astype('uint16')

In [9]:
fifth = train.shape[0] / 5
x_validation = train.loc[:fifth, :]
x_train = train.loc[fifth:,:]
del train
gc.collect()

94

In [10]:
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
        'max_depth': -1,  # -1 means no limit
        'min_child_samples': 20,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.6,  # Subsample ratio of the training instance.
        'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 8,
        'verbose': 0,
    }


In [11]:
print(x_train.columns)
x_train.loc[:, 'is_attributed'].values

Index(['app', 'device', 'os', 'channel', 'is_attributed', 'hour', 'day',
       'n_channels'],
      dtype='object')


array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [12]:
lgb_train = lgb.Dataset(x_train.loc[x_train.columns != 'is_attributed', :].values, label=x_train.loc[:,'is_attributed'].values
                          )
lgb_valid = lgb.Dataset(x_validation.loc[x_validation.columns != 'is_attributed', :].values, label=x_validation.loc[:,'is_attributed'].values
                          )


In [13]:
evals_results = {}

In [14]:
bst1 = lgb.train(lgb_params, 
                     lgb_train, 
                     valid_sets=[ lgb_valid], 
                     valid_names=['valid'], 
                     evals_result=evals_results, 
                     num_boost_round=10,
                     early_stopping_rounds=10,
                     verbose_eval=10)

LightGBMError: b'Cannot construct Dataset since there are not useful features.                 It should be at least two unique rows.                 If the num_row (num_data) is small, you can set min_data=1 and min_data_in_bin=1 to fix this.                 Otherwise please make sure you are using the right dataset.'

In [26]:
submit = pd.read_csv(path + "test.csv", dtype='int', usecols=['click_id'])
submit['is_attributed'] = res

In [27]:
submit.to_csv('submission.csv', index=False)